# Objective:

Aggregate precipitation increment data from the NRCS database.

#### Sources: 
- NRCS interactive map: https://www.nrcs.usda.gov/wps/portal/wcc/home/
- NRCS web report scripting (URL-to-File method): https://www.wcc.nrcs.usda.gov/web_service/NWCC_Web_Report_Scripting.txt

NRCS interactive map provides precipitation increments. For SNOTEL stations, snow-adjusted precipitation increments are also available. Note that the database through the map was missing precipitation records of a few stations. The web report scripting was used to access those records.

In [48]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os, glob
import geopandas as gpd
from shapely.geometry import Point

# Metadata

Downloaded using the NRCS interactive map

In [14]:
Metadata = pd.read_csv('../data_raw/NRCS_metadata.csv', skiprows=261, dtype=str)

# Snow-adjusted precipitation
Downloaded from the NRCS interactive map

#### Read in Snow-adjusted precipitation

In [3]:
PRCP_SA = pd.read_csv('../data_raw/NRCS_data_PRCP_inc_SnowAdj.csv', skiprows=264, dtype=str)

## Extract precip values and ignore QA/QC flags
PRCP_SA_val = PRCP_SA.iloc[:,1::3]

## Drop columns that have all nan values
PRCP_SA_val = PRCP_SA_val.dropna(axis=1, how='all')

## Rename columns of dataframes by using information from metadata.

PRCP_SA_val_full = PRCP_SA.iloc[:,1::3]
filled_indices_SA = [PRCP_SA_val_full.columns.get_loc(PRCP_SA_val.columns[i]) for i in np.arange(PRCP_SA_val.columns.size)]

PRCP_SA_val.columns = Metadata.iloc[filled_indices_SA,0]
PRCP_SA_val.columns.name=''

#### Note that all snow-adjusted records belong to SNOTEL (SNTL) stations. 

In [4]:
Metadata.iloc[filled_indices_SA,2].values

array(['SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL',
       'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL',
       'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL',
       'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL',
       'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL',
       'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL',
       'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL',
       'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL',
       'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL',
       'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL',
       'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL',
       'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL',
       'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL',
       'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL', 'SNTL',
      

# Non-snow adjusted precipitation
Downloaded from the NRCS interactive map

#### Read in data only for non-SNOTEL stations.

In [5]:
non_SNTL_ind = Metadata[Metadata['Network Code'] != 'SNTL'].index

PRCP_inc = pd.read_csv('../data_raw/NRCS_data_PRCP_inc.csv', skiprows=264, dtype=str)

## Extract precip values and ignore QA/QC flags
PRCP_inc_val = PRCP_inc.iloc[:,1::3]

## Keep only the non-SNOTEL columns
PRCP_inc_val = PRCP_inc_val.iloc[:,non_SNTL_ind]

## Rename columns
PRCP_inc_val.columns = Metadata[Metadata['Network Code'] != 'SNTL']['Station Id'].values

## Drop columns that have all nan values
PRCP_inc_val = PRCP_inc_val.dropna(axis=1, how='all')

## Drop the second column (the time series only has one value)
PRCP_inc_val = PRCP_inc_val.drop(columns = PRCP_inc_val.columns[1])

# Merge the two types of precipitation data in a single dataframe

In [6]:
inc_date = lambda x: (datetime.datetime.strptime(x,"%Y-%m-%d") + datetime.timedelta(1)).strftime("%Y-%m-%d")

PRCP_SA_val_float = PRCP_SA_val.astype('float')
PRCP_inc_val_float = PRCP_inc_val.astype('float')

NRCS_data = pd.merge(PRCP_SA['Date'], PRCP_SA_val_float, left_index=True, right_index=True)
NRCS_data = pd.merge(NRCS_data, PRCP_inc_val_float, left_index=True, right_index=True)

#### Increment dates
All the precipitation values obtained from the database are daily values (24h starting at midnight local time). The dates are incremented by 1 as a personal preference, so that precipitation for each day corresponds to the previous 24h. 

In [7]:
incremented_dates = [inc_date(x) for x in NRCS_data['Date']]
NRCS_data['Date'] = incremented_dates
NRCS_data = NRCS_data.iloc[:-1,:]

## convert datatype to float, and multiply by 25.4 to convert inches to mm.
NRCS_data.iloc[:,1:] = NRCS_data.iloc[:,1:]*25.4

#### Add columns for Year, Month and Day to dataframe

In [8]:
date_stripped = [datetime.datetime.strptime(item, "%Y-%m-%d") for item in NRCS_data['Date']]

years = [item.year for item in date_stripped]
months = [item.month for item in date_stripped]
days = [item.day for item in date_stripped]

Date_df = pd.DataFrame({'Date':NRCS_data['Date'], 'Year': years, 'Month': months, 'Day':days})

In [9]:
NRCS_df = pd.merge(Date_df, NRCS_data)

# Additional COOP network stations

These were downloaded using NRCS web-report scripting. The precipitation data for these stations were missing from the data obtained using the NRCS interactive map.

In [10]:
def clean_NRCSdata(prec):
    index = prec.index[prec.values == -99.90]
    if len(index):
        for val in index:
            prec.iloc[val] = np.nan
        return


def get_data_additional_stations(StationID):
    datapath = '../data_raw/Additional_Stations/'
    filenames=[]
    for file in glob.glob( os.path.join(datapath + StationID +'*WY.csv') ):
        filenames.append(file)
    filenames.sort()
    Date = []
    daily_prec = []
    for file in filenames:
        data = pd.read_csv(file, skiprows=3)
        if len(data):
            prec = data.loc[:,'PREC.I-1 (in) '].copy()
            clean_NRCSdata(prec)
            Date.append(data.loc[:,'Date'].iloc[0:-1])
            daily_prec.append((prec.iloc[1:].values - prec.iloc[0:-1].values)*25.4)
        
    Date = pd.concat(Date, ignore_index=True)
    daily_prec = np.concatenate(daily_prec)

    incremented_dates = [inc_date(x) for x in Date]   
    return pd.DataFrame({'Date':incremented_dates, StationID:daily_prec})

In [11]:
additional_stations = ['0484', '0761', '2192', '0951', '0909']

for item in additional_stations:
    data = get_data_additional_stations(item)
    NRCS_df = pd.merge(NRCS_df, data, how='left')

# Save precipitation records as csv file

In [12]:
NRCS_df.to_csv('../data_processed/NRCS_dates_2008_2017.csv', index=False)

# Process Metadata

In [67]:
Metadata = pd.read_csv('../data_raw/NRCS_metadata.csv', skiprows=261, dtype=str)

In [68]:
Metadata = Metadata[Metadata['Station Id'].isin(NRCS_df.columns[4:])]

Metadata = Metadata[['Station Id', 'Elevation', 'Latitude', 'Longitude']]

Metadata.rename(columns={'Station Id':'StationID'}, inplace=True, copy=True)

Metadata.reset_index(drop=True, inplace=True)

Metadata[['Elevation','Latitude','Longitude']] = Metadata[['Elevation','Latitude','Longitude']].apply(pd.to_numeric)

In [69]:
Metadata

,StationID,Elevation,Latitude,Longitude
0,2138,6445,37.67455,-109.36429
1,1030,10960,40.35098,-106.38142
2,0484,6240,41.03333,-107.65000
3,317,7440,41.05413,-107.26609
4,1061,9080,40.06153,-107.00955
...,...,...,...,...
147,859,8950,41.00289,-106.90848
148,864,8641,39.96450,-110.98845
149,869,9540,40.34703,-106.09433
150,874,11000,37.47922,-106.80170


# Save processed Metadata to csv

In [71]:
Metadata.to_csv('../data_processed/Metadata_processed.csv', index=False)